In [1]:
#Imports
import pandas as pd
from datetime import datetime
import csv

#Sqlalchemy imports
from sqlalchemy import Column, Float, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,inspect,func
# Flask imports
from flask import Flask, jsonify, render_template

In [2]:
#######Reimport csv, examine, and reoutput to sqlite#######
#CSV import
clean_no_index = "../CleanData/clean_no_index.csv"
clean_no_index_df = pd.read_csv(clean_no_index,encoding='latin-1')

In [3]:
#Data structure overview
print("clean_testcolumn information: \n")
print(clean_no_index_df.columns,clean_no_index_df.dtypes) 

clean_testcolumn information: 

Index(['restaurant_name', 'address', 'city', 'state', 'zip_code', 'phone',
       'cuisine_type', 'rating', 'price', 'latitude', 'longitude'],
      dtype='object') restaurant_name     object
address             object
city                object
state               object
zip_code            object
phone              float64
cuisine_type        object
rating             float64
price               object
latitude           float64
longitude          float64
dtype: object


In [4]:
#Examine header
clean_no_index_df.head()

,restaurant_name,address,city,state,zip_code,phone,cuisine_type,rating,price,latitude,longitude
0,Blue Collar,6730 Biscayne Blvd,Miami,FL,33138,1.305756e+10,American (Traditional),4.0,$$,26.079674,-80.252222
1,Atlantic No 5,605 W Main St,Louisville,KY,40202,1.502883e+10,Breakfast & Brunch,4.0,$$,25.750963,-80.258375
2,Famous Dave's,17770 Southcenter Pkwy,Tukwila,WA,98188,1.206631e+10,Barbeque,3.0,$$,34.028292,-81.093276
3,Quiznos,1595 N Peach Ave,Fresno,CA,93727,1.559455e+10,Fast Food,3.0,$,44.939642,-93.136030
4,Hobbit Cafe,2243 Richmond Ave,Houston,TX,77098,1.713527e+10,Breakfast & Brunch,3.5,$$,40.115749,-83.089461


In [5]:
print(clean_no_index_df.index.name)

None


In [6]:
clean_no_index_df['id']=clean_no_index_df.index
clean_no_index_df = clean_no_index_df.reset_index(drop=True)

In [7]:
print(clean_no_index_df.columns,clean_no_index_df.dtypes) 

Index(['restaurant_name', 'address', 'city', 'state', 'zip_code', 'phone',
       'cuisine_type', 'rating', 'price', 'latitude', 'longitude', 'id'],
      dtype='object') restaurant_name     object
address             object
city                object
state               object
zip_code            object
phone              float64
cuisine_type        object
rating             float64
price               object
latitude           float64
longitude          float64
id                   int64
dtype: object


In [8]:
clean_no_index_df.head()

,restaurant_name,address,city,state,zip_code,phone,cuisine_type,rating,price,latitude,longitude,id
0,Blue Collar,6730 Biscayne Blvd,Miami,FL,33138,1.305756e+10,American (Traditional),4.0,$$,26.079674,-80.252222,0
1,Atlantic No 5,605 W Main St,Louisville,KY,40202,1.502883e+10,Breakfast & Brunch,4.0,$$,25.750963,-80.258375,1
2,Famous Dave's,17770 Southcenter Pkwy,Tukwila,WA,98188,1.206631e+10,Barbeque,3.0,$$,34.028292,-81.093276,2
3,Quiznos,1595 N Peach Ave,Fresno,CA,93727,1.559455e+10,Fast Food,3.0,$,44.939642,-93.136030,3
4,Hobbit Cafe,2243 Richmond Ave,Houston,TX,77098,1.713527e+10,Breakfast & Brunch,3.5,$$,40.115749,-83.089461,4


In [9]:
#Output clean test data to CSV [redundant - but do not delete]
clean_no_index_df.to_csv("../CleanData/clean_test.csv", index=True, header=True)


In [10]:
#Create engine
engine = create_engine("sqlite:///../sqldata/clean_test.db",echo=True) #Set echo=True for debugging
inspector = inspect(engine)
Base = declarative_base()


2018-03-24 02:46:48,190 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-24 02:46:48,192 INFO sqlalchemy.engine.base.Engine ()
2018-03-24 02:46:48,195 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-24 02:46:48,196 INFO sqlalchemy.engine.base.Engine ()


In [11]:
#clean_no_index.csv ---> dataframe ---> to_dict(orient='records') --> sqlite
def populate(engine, table, csvfile):
    conn = engine.connect()
    df =pd.read_csv(csvfile,encoding='latin-1')
    data = df.to_dict(orient='records')
    conn.execute(table.insert(), data)

#Establish class base
class Vegetarian(Base):
    __tablename__ = "vegetarian"
    id = Column(Integer, primary_key=True)
    restaurant_name = Column(String)
    address = Column(String)
    city = Column(String)
    state = Column(String)
    zip_code = Column(Integer)
    phone = Column(Integer)
    cuisine_type = Column(String)
    rating = Column(String)
    price = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    
#run populate function on sqlite db
populate(engine,Vegetarian.__table__,"../CleanData/clean_test.csv")

2018-03-24 02:46:48,744 INFO sqlalchemy.engine.base.Engine INSERT INTO vegetarian (id, restaurant_name, address, city, state, zip_code, phone, cuisine_type, rating, price, latitude, longitude) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2018-03-24 02:46:48,760 INFO sqlalchemy.engine.base.Engine ((0, 'Blue Collar', '6730 Biscayne Blvd', 'Miami', 'FL', '33138', 13057560366.0, 'American (Traditional)', 4.0, '$$', 26.07967414, -80.25222212), (1, 'Atlantic No 5', '605 W Main St', 'Louisville', 'KY', '40202', 15028833398.0, 'Breakfast & Brunch', 4.0, '$$', 25.750963, -80.258375), (2, "Famous Dave's", '17770 Southcenter Pkwy', 'Tukwila', 'WA', '98188', 12066312000.0, 'Barbeque', 3.0, '$$', 34.02829213, -81.09327574), (3, 'Quiznos', '1595 N Peach Ave', 'Fresno', 'CA', '93727', 15594548477.0, 'Fast Food', 3.0, '$', 44.939642, -93.13603), (4, 'Hobbit Cafe', '2243 Richmond Ave', 'Houston', 'TX', '77098', 17135265460.0, 'Breakfast & Brunch', 3.5, '$$', 40.11574901, -83.08946112), (5, 'Pizzeria 330

In [12]:
conn = engine.connect()
Base.metadata.create_all(engine)
session = Session(bind=engine)

#clean_no_index_df.to_sql('vegetarian', conn, if_exists='replace', index=False)

2018-03-24 02:46:48,893 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("vegetarian")
2018-03-24 02:46:48,894 INFO sqlalchemy.engine.base.Engine ()


In [13]:
#Verify table creation with inspector + verify table information:
print(inspector.get_table_names())
data = engine.execute("PRAGMA table_info([vegetarian]);")
for item in data:
    print(item)

2018-03-24 02:46:48,909 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-03-24 02:46:48,910 INFO sqlalchemy.engine.base.Engine ()
['vegetarian']
2018-03-24 02:46:48,917 INFO sqlalchemy.engine.base.Engine PRAGMA table_info([vegetarian]);
2018-03-24 02:46:48,919 INFO sqlalchemy.engine.base.Engine ()
(0, 'id', 'INTEGER', 1, None, 1)
(1, 'restaurant_name', 'VARCHAR', 0, None, 0)
(2, 'address', 'VARCHAR', 0, None, 0)
(3, 'city', 'VARCHAR', 0, None, 0)
(4, 'state', 'VARCHAR', 0, None, 0)
(5, 'zip_code', 'INTEGER', 0, None, 0)
(6, 'phone', 'INTEGER', 0, None, 0)
(7, 'cuisine_type', 'VARCHAR', 0, None, 0)
(8, 'rating', 'VARCHAR', 0, None, 0)
(9, 'price', 'VARCHAR', 0, None, 0)
(10, 'latitude', 'FLOAT', 0, None, 0)
(11, 'longitude', 'FLOAT', 0, None, 0)


In [14]:
#columns = inspector.get_columns('vegetarian')
#print("Column names:")
#for c in columns:
    #print(c['name'])
#print("========================")
#print (columns)

In [15]:
Base.metadata.create_all(engine)
session.commit()

2018-03-24 02:46:48,945 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("vegetarian")
2018-03-24 02:46:48,947 INFO sqlalchemy.engine.base.Engine ()


In [16]:
#verify table information + column properties:
data = engine.execute("PRAGMA table_info([vegetarian]);")
for item in data:
    print(item)

2018-03-24 02:46:48,958 INFO sqlalchemy.engine.base.Engine PRAGMA table_info([vegetarian]);
2018-03-24 02:46:48,959 INFO sqlalchemy.engine.base.Engine ()
(0, 'id', 'INTEGER', 1, None, 1)
(1, 'restaurant_name', 'VARCHAR', 0, None, 0)
(2, 'address', 'VARCHAR', 0, None, 0)
(3, 'city', 'VARCHAR', 0, None, 0)
(4, 'state', 'VARCHAR', 0, None, 0)
(5, 'zip_code', 'INTEGER', 0, None, 0)
(6, 'phone', 'INTEGER', 0, None, 0)
(7, 'cuisine_type', 'VARCHAR', 0, None, 0)
(8, 'rating', 'VARCHAR', 0, None, 0)
(9, 'price', 'VARCHAR', 0, None, 0)
(10, 'latitude', 'FLOAT', 0, None, 0)
(11, 'longitude', 'FLOAT', 0, None, 0)


In [17]:
Base.metadata.create_all(engine)
session.commit()

2018-03-24 02:46:48,973 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("vegetarian")
2018-03-24 02:46:48,975 INFO sqlalchemy.engine.base.Engine ()


In [18]:
#Move testing_grounds.ipynb after response working:
## Display the row's columns and data in dictionary format
first_row = session.query(Vegetarian).all()
first_row.__dict__

2018-03-24 02:46:49,013 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-24 02:46:49,017 INFO sqlalchemy.engine.base.Engine SELECT vegetarian.id AS vegetarian_id, vegetarian.restaurant_name AS vegetarian_restaurant_name, vegetarian.address AS vegetarian_address, vegetarian.city AS vegetarian_city, vegetarian.state AS vegetarian_state, vegetarian.zip_code AS vegetarian_zip_code, vegetarian.phone AS vegetarian_phone, vegetarian.cuisine_type AS vegetarian_cuisine_type, vegetarian.rating AS vegetarian_rating, vegetarian.price AS vegetarian_price, vegetarian.latitude AS vegetarian_latitude, vegetarian.longitude AS vegetarian_longitude 
FROM vegetarian
2018-03-24 02:46:49,022 INFO sqlalchemy.engine.base.Engine ()


AttributeError: 'list' object has no attribute '__dict__'